## Data loading

In [1]:
from clef.utils.data_loading import load_datasets
import os

root_path = '../../'
out_dir = './data-out/setup1'

train, dev = load_datasets(preprocess=True,
                           add_author_name=True,
                           add_author_bio=False,
                           root_path= root_path,)

# ensure out_dir directories exist for later
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    if not os.path.exists(os.path.join(out_dir, 'eval')):
        os.makedirs(os.path.join(out_dir, 'eval'))

loaded 96 training json lines and 32 dev json lines.


In [3]:
# we fake similarity scores for RQ2

import json
from copy import deepcopy

rq2_dataset = deepcopy(dev)

for i, item in enumerate(rq2_dataset):
    item['retrieved_evidence'] = []

    for author_account, tweet_id, tweet_text in item['evidence']:
        item['retrieved_evidence'] += [[
            author_account, #'author_account'
            tweet_id, #'authority_tweet_id'
            tweet_text, #'doc_text'
            1, #'rank'
            1, #'score'
        ]]

    rq2_dataset[i] = item
print(json.dumps(rq2_dataset[0], indent=2))

{
  "id": "AuRED_142",
  "rumor": "Naturalization decree in preparation: Lebanese passports for sale !",
  "label": "REFUTES",
  "timeline": [
    [
      "https://twitter.com/LBpresidency",
      "1556600039211925504",
      "Account: Lebanese Presidency\nText: Today the President of the Republic General Michel Aoun signed 9 laws that were previously approved by the House of Representatives Details at the following link:"
    ],
    [
      "https://twitter.com/LBpresidency",
      "1556559119045332992",
      "Account: Lebanese Presidency\nText: President Aoun received the Minister of Foreign Affairs and Expatriates Dr Abdullah Bouhabib and the Minister of Social Affairs Hector Hajjar and discussed with them developments related to the file of displaced Syrians in Lebanon"
    ],
    [
      "https://twitter.com/LBpresidency",
      "1556558220533157890",
      "Account: Lebanese Presidency\nText: President Aoun received the new British Ambassador to Lebanon Hamish Cowell and discuss

### RQ2 NLI

In [4]:
from clef.verification.verify import check_dataset_with_model
from clef.utils.data_loading import write_jsonlines_from_dicts

result = check_dataset_with_model(rq2_dataset, 'bart', debug=True)

outfile = f'{out_dir}/zeroshot-ver-rq2-nli.jsonl'
write_jsonlines_from_dicts(outfile, result)

  0%|          | 0/32 [00:00<?, ?it/s]

Naturalization decree in preparation: Lebanese passports for sale !
	0.6264864802360535 Account: Lebanese Presidency Text: The Information Office of the Presidency of the Republic denies a false news broadcast by the MTV station about Baabda Palace preparing a decree naturalizing 4 000 people and recalls that it had denied yesterday the false information published by the French magazine Liberation about the same fabricated news
	0.6944529414176941 Account: Lebanese Presidency Text: The Information Office of the Presidency of the Republic: What was published by the French newspaper Liberation about the selling of Lebanese passports to non-Lebanese is false and baseless news
label: REFUTES
predicted: SUPPORTS

In the video The spread of unidentified gunmen east of the capital Baghdad
	0.6185822486877441 Account: Security Media Cell Text: The security forces of all types and formations are the ones who hold the reins in all governorates of the country and there is no truth to what some so

### RQ2 openai

In [4]:
from clef.verification.verify import check_dataset_with_model
from clef.utils.data_loading import write_jsonlines_from_dicts

result = check_dataset_with_model(rq2_dataset, 'openai', debug=True)

outfile = f'{out_dir}/zeroshot-ver-rq2-openai.jsonl'
write_jsonlines_from_dicts(outfile, result)

  0%|          | 0/32 [00:00<?, ?it/s]

Naturalization decree in preparation: Lebanese passports for sale !
	-0.95 Account: Lebanese Presidency Text: The Information Office of the Presidency of the Republic denies a false news broadcast by the MTV station about Baabda Palace preparing a decree naturalizing 4 000 people and recalls that it had denied yesterday the false information published by the French magazine Liberation about the same fabricated news
	-0.95 Account: Lebanese Presidency Text: The Information Office of the Presidency of the Republic: What was published by the French newspaper Liberation about the selling of Lebanese passports to non-Lebanese is false and baseless news
label: REFUTES
predicted: REFUTES

In the video The spread of unidentified gunmen east of the capital Baghdad
	-0.95 Account: Security Media Cell Text: The security forces of all types and formations are the ones who hold the reins in all governorates of the country and there is no truth to what some social media sites have reported about the

## Evaluation

In [5]:
from clef.utils.scoring import eval_run_custom
from clef.utils.data_loading import task5_dir

import pandas as pd
from IPython.core.display import display_html

import datetime
from csv import writer

sample_submission_file = f'{root_path}/{task5_dir}/submission_samples/KGAT_zeroShot_verification_English_dev.json'

rq2_nli_submission_file = f'{out_dir}/zeroshot-ver-rq2-nli.jsonl'
rq2_openai_submission_file = f'{out_dir}/zeroshot-ver-rq2-openai.jsonl'

ground_truth_file = f'{root_path}/{task5_dir}/data/English_dev.json'

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8



eval RQ2

In [7]:
time_now  = datetime.datetime.now().strftime('%d_%m_%Y_%H_%M_%S') 
out_file = f'{out_dir}/eval/RQ2-{time_now}.csv'

eval_data = [
    ['baseline',    *eval_run_custom(sample_submission_file, ground_truth_file, out_file)],
    ['RQ2-nl',      *eval_run_custom(rq2_nli_submission_file, ground_truth_file, out_file)],
    ['RQ2-openai',  *eval_run_custom(rq2_openai_submission_file, ground_truth_file, out_file)],
]

eval_df = pd.DataFrame(eval_data)
eval_df.columns = ['method', 'macro-F1', 'strict-macro-F1']

df_r5  = eval_df[['method', 'macro-F1']].sort_values('macro-F1', axis=0, ascending=False)
df_map = eval_df[['method', 'strict-macro-F1']].sort_values('strict-macro-F1', axis=0, ascending=False)

df1_styler = df_r5.style.set_table_attributes("style='display:inline'").set_caption('macro-F1')
df2_styler = df_map.style.set_table_attributes("style='display:inline'").set_caption('strict-macro-F1')

eval_df.to_csv(out_file)

display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw=True)

,method,macro-F1
2,RQ2-openai,0.958333
0,baseline,0.508159
1,RQ2-nl,0.371429
,method,strict-macro-F1
2,RQ2-openai,0.958333
0,baseline,0.508159
1,RQ2-nl,0.371429
